In [187]:
import pandas as pd
import numpy as np

In [218]:
df_all = pd.read_excel('../data/AIMS All Docs and Mnt Adapt Focused.xlsx', sheetname='All Docs', header=1)
df_mtn = pd.read_excel('../data/AIMS All Docs and Mnt Adapt Focused.xlsx', sheetname='Mnt Adaptations', header=1)

In [219]:
data = df_all.copy()
data['Q 11.'] = data['Q 11.'].str.replace(',', ';')
exclusions = ['Blue = Peru', 'Green = Nepal', 'Orange = Both', 'Red = manually updated']
data = data[-data['Q 11.'].isin(exclusions)]

countries = data['Q 11.'].str.split(';', expand=True)

def observation_count(data, subset='AllStudies', level='observations'):
    """
    Counts the number of times an ISO code is referenced
    As per observations
    params: data - pandas dataframe
    params: subset - how to subset the data ('PR', 'GL')
    """
    if subset is not 'AllStudies':
        data = data.loc[data['Dcoument Type'] == subset]
    else:
        pass
    
    data['Q 11.'] = data['Q 11.'].str.replace(',', ';')
    exclusions = ['Blue = Peru', 'Green = Nepal', 'Orange = Both', 'Red = manually updated']
    data = data[-data['Q 11.'].isin(exclusions)]
    
    
    if level is 'observations':

        countries = data['Q 11.'].str.split(';', expand=True)
        countries = pd.DataFrame(countries.as_matrix().ravel())
        countries.columns = ['ISO']
        countries['ISO'] = countries['ISO'].str.strip()
        countries = pd.DataFrame(countries['ISO'].value_counts()).reset_index()
        countries.columns = ['ISO', 'Count']
        countries = countries.dropna()
        countries = countries.loc[countries['ISO'] !='nan']

        
    else:
        
        countries = data['Q 11.'].str.split(';', expand=True)
        countries['DOI/UID'] = data['DOI/UID']

        for c in countries.columns[:-1]:
            countries[c] = countries[c].astype(str) + '_' + countries['DOI/UID'].astype(str)
            countries[c] = np.where(-countries[c].str.contains('None'), countries[c], np.nan)
            countries[c] = countries[c].str.strip()

        countries = pd.DataFrame(np.unique(countries.as_matrix().ravel()))
        countries = countries[0].str.split('_', expand=True)
        countries = countries.dropna()
        countries.columns = ['ISO', 'DOI']    
        countries['ISO'] = countries['ISO'].str.strip()
        countries = pd.DataFrame(countries['ISO'].value_counts()).reset_index()
        countries.columns = ['ISO', 'Count']
        countries = countries.dropna()
        countries = countries.loc[countries['ISO'] !='nan']
        
    return countries

In [222]:
subsets = ['PR', 'GL', 'AllStudies']
levels = ['observations', 'studies']
datasets = {'All': df_all, 
            'Mountain': df_mtn}

for name, dataset in datasets.iteritems():

    for subset in subsets:
        
        for level in levels:
            
            out = observation_count(dataset, subset=subset, level=level)
            out = out.set_index('ISO')
            out.to_csv('../data/country_counts_{}_{}_{}.csv'.format(name, subset, level))
            
            print len(out), name, subset, level

/Users/Vinny_Ricciardi/virtenvs/py2_geocode_update/lib/python2.7/site-packages/ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


21 Mountain PR observations
21 Mountain PR studies
15 Mountain GL observations
15 Mountain GL studies
27 Mountain AllStudies observations
27 Mountain AllStudies studies
33 All PR observations
33 All PR studies
28 All GL observations
28 All GL studies
42 All AllStudies observations
42 All AllStudies studies
